# PyFtdi test
http://eblot.github.io/pyftdi/  
https://pypi.org/project/pyftdi/  
https://github.com/eblot/pyftdi  
https://www.ftdichip.com/  
https://www.ftdichip.com/Products/ICs/FT232H.htm  

Pinout:  
http://eblot.github.io/pyftdi/pinout.html  

OLED display:  
https://circuitdigest.com/article/ssd1306-oled-display  

### 需安裝

```
pip install pyusb

```

### Windows DLL
[libusb-win32-devel-filter-1.2.6.0.exe](https://sourceforge.net/projects/libusb-win32/files/libusb-win32-releases/1.2.6.0/)  

https://sourceforge.net/projects/libusb-win32/  



### Linux 權限
https://ubuntuforums.org/showthread.php?t=1682084  
https://linuxconfig.org/tutorial-on-how-to-write-basic-udev-rules-in-linux  
```
If anyone is interested.. fixed with udev rule:

cd /etc/udev/rules.d/

sudo nano 70-usb.rules

SUBSYSTEM=="usb", ATTRS{idVendor}=="1a86", GROUP="dialout", MODE="666" 
SUBSYSTEM=="usb", ATTRS{idVendor}=="0403", GROUP="dialout", MODE="666" 

then reloaded udev rules:

sudo udevadm control --reload-rules  
sudo udevadm trigger

```

### 安裝步驟

- 安裝 [libusb-win32-devel-filter](https://sourceforge.net/projects/libusb-win32/)
    - [libusb-win32-devel-filter-1.2.6.0.exe](https://sourceforge.net/projects/libusb-win32/files/libusb-win32-releases/1.2.6.0/) 
- 使用 libUSB-Win32, install all class filters    


- 安裝 PySerial
```
pip install pyserial
```
    
- 安裝 PyUSB
```
pip install pyusb
``` 

- 安裝 PyFtdi (http://eblot.github.io/pyftdi/installation.html)
```
pip install pyftdi
``` 

In [1]:
%pylab inline  

import os
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'codes')))
sys.path.append(os.path.abspath(os.path.join('..', 'codes', 'test', 'lib', 'adafruit', 'display')))

import pyftdi

from pprint import pprint

Populating the interactive namespace from numpy and matplotlib


## I2C test

In [ ]:
from bridges.ftdi.controllers.i2c import I2cController

ctrl = I2cController()
i2c = ctrl.get_i2c()
# i2c.scan()


from display_ssd1306_i2c import Display

d = Display(i2c)  # d = Display(smbus)

d.clear()

d.rect(0, 0, 30, 30, 56)
d.fill_rect(0, 0, 30, 30, 56)
d.hline(0, 0, 60, 56)
d.vline(0, 0, 60, 56)
d.line(0, 0, 60, 60, 56)
d.pixel(40, 40, 56)
d.show()

ctrl.terminate()

## SPI test
http://eblot.github.io/pyftdi/api/spi.html  

In [3]:
from bridges.ftdi.controllers.spi import SpiController 

ctrl = SpiController()
spi = ctrl.get_spi(cs = 0, freq = 12E6, mode = 0)


import bridges.ftdi.adapters.micropython.machine as machine
from display_ssd1306_spi import Display

gpio = ctrl.get_gpio() 

dc = gpio.Pin('ADBUS4', mode = machine.Pin.OUT)
res = gpio.Pin('ADBUS5', mode = machine.Pin.OUT)
cs = gpio.Pin('ADBUS7', mode = machine.Pin.OUT)

d = Display(spi, dc, res, cs)
d.clear()
d.rect(0, 0, 30, 30, 56)
d.fill_rect(0, 0, 30, 30, 56)
d.hline(0, 0, 60, 56)
d.vline(0, 0, 60, 56)
d.line(0, 0, 60, 60, 56)
d.pixel(40, 40, 56)
d.show()

ctrl.terminate()

## SpiGpioPort test
http://eblot.github.io/pyftdi/api/spi.html#pyftdi.spi.SpiGpioPort  

In [4]:
import bridges.ftdi.adapters.micropython.machine as machine
from bridges.ftdi.controllers.spi import SpiController 
from time import sleep

ctrl = SpiController()
spi = ctrl.get_spi(cs = 0, freq = 12E6, mode = 0)
gpio = ctrl.get_gpio()

pin_name = 'ADBUS6'
pin = gpio.Pin(pin_name, mode = machine.Pin.OUT)

# pin_idx = gpio.get_pin_idx(pin_name)
# pin = gpio.Pin(7, mode = machine.Pin.OUT)


def blinks():
    from time import sleep

    def blink(delay = 0.2):
        pin.toggle()
        sleep(delay)
        pin.toggle()
        sleep(delay)


    for i in range(3):
        print(i)
        blink()
        

blinks()
    
ctrl.terminate() 

0
1
2


## GpioPort test
http://eblot.github.io/pyftdi/api/gpio.html  

In [5]:
import bridges.ftdi.adapters.micropython.machine as machine
from bridges.ftdi.controllers.gpio import GpioController


ctrl = GpioController()
# print(gpio.addressable_pins)

pin_in = ctrl.Pin('ADBUS5', mode = machine.Pin.IN)
pin_out = ctrl.Pin('ADBUS6', mode = machine.Pin.OUT)

print('pin_in:', pin_in())

def blinks():
    from time import sleep

    def blink(delay = 0.2):
        pin_out.toggle()
        sleep(delay)
        pin_out.toggle()
        sleep(delay)


    for i in range(3):
        print(i)
        blink()



blinks()

ctrl.terminate()

pin_in: 1
0
1
2


## UART test
http://eblot.github.io/pyftdi/api/uart.html  

In [6]:
from bridges.ftdi.controllers.uart import UartController

ctrl = UartController()
# port = ctrl.Serial(baudrate = 115200)
# port = ctrl.Serial(baudrate = 115200, bytesize=8, parity='N', stopbits=1)
port = ctrl.UART(baudrate = 9600, bits = 8, parity = None, stop = 1)

# port = ctrl.UART()
port.write(b'Hello World')

# data = port.read(1024)


ctrl.terminate()

port

UART<id=0x22a68489d68, open=False>(port='ftdi://ftdi:ft232h::/1', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)